Following tutorial : https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
aa1data = sc.read_10x_mtx(
    '../data/Huetal2022/AA_patient_1/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading


In [ ]:
aa1data.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`

In [ ]:
aa1data.obs

In [ ]:
aa2data = sc.read_10x_mtx(
    '../data/Huetal2022/AA_patient_2/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)

aa2data.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`   
aa2data 
aa2data.obs

In [ ]:
n1data = sc.read_10x_mtx(
    '../data/Huetal2022/N_patient_1/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)

n1data.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`   
n1data.obs

In [ ]:
n2data = sc.read_10x_mtx(
    '../data/Huetal2022/N_patient_2/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)

n2data.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`   
n2data 

In [ ]:
total = ad.concat([aa1data, aa2data, n1data, n2data], merge="same", label="dataset")
total.var_names_make_unique()
total.obs

In [ ]:
celltypes = sc.read_text('../data/Huetal2022/celltyping_results.txt', first_column_names=True, delimiter='	', dtype=str)
# celltypes.obs

In [ ]:
total = ad.concat([total, celltypes], axis=1)

In [ ]:
sc.pp.filter_cells(total, min_genes=200)
sc.pp.filter_genes(total, min_cells=3)

In [ ]:
total

In [ ]:
total.layers["counts"] = total.X.copy() #this is needed for scvi later

In [ ]:
sc.pp.normalize_total(total, target_sum=1e4)

In [ ]:
sc.pp.log1p(total)

In [ ]:
sc.pp.highly_variable_genes(total, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(total)

In [ ]:
total.raw = total

In [ ]:
total = total[:, total.var.highly_variable]

In [ ]:
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(total, svd_solver='arpack')

In [ ]:
sc.pl.pca(total, color='dataset')

In [ ]:
sc.pl.pca_variance_ratio(total, log=False)

In [ ]:
sc.pp.neighbors(total, n_neighbors=10, n_pcs=50)

In [ ]:
sc.tl.umap(total)

In [ ]:
sc.pl.umap(total, color=['dataset'])

---

SCVI on total as count matrix

In [ ]:
import scvi

In [ ]:
scvi.model.SCVI.setup_anndata(total, layer="counts", categorical_covariate_keys=["dataset"])

In [ ]:
vae = scvi.model.SCVI(total)
vae

In [ ]:
vae.train()

In [ ]:
vae

In [ ]:
latent = vae.get_latent_representation()
total.obsm["X_scvi"] = latent

In [ ]:
sc.pp.neighbors(total, use_rep="X_scvi")
total.obsp["distances"]

In [ ]:
sc.tl.umap(total)
sc.pl.umap(total, color=['dataset'])

In [ ]:
total.obsm["X_normalized_scVI"] = vae.get_normalized_expression()
total.obsm["X_normalized_scVI"]

Differential expression between individuals

In [ ]:
diffexpdf = vae.differential_expression(total, )
diffexpdf

In [ ]:
vae.save("vae_SCVI_trained_with_donor_as_covariate")

In [ ]:
aa_subset = ad.concat([total[total.obs.dataset == "0"], total[total.obs.dataset == "1"]])
aa_subset
latent_aa_subset = vae.get_latent_representation(aa_subset)

In [ ]:
h_subset = ad.concat([total[total.obs.dataset == "2"], total[total.obs.dataset == "3"]])
h_subset
latent_h_subset = vae.get_latent_representation(h_subset)

Demonstrate presence of nuisance variation 

In [ ]:
# run PCA then generate UMAP plots
sc.tl.pca(total)
sc.pp.neighbors(total, n_pcs=30, n_neighbors=20)
sc.tl.umap(total, min_dist=0.3)

In [ ]:
sc.pl.umap(
    total,
    color=["dataset"],
    frameon=False,
)

Batch corrected visualization

In [ ]:

total.obsm["X_scVI"] = latent

# use scVI latent space for UMAP generation
sc.pp.neighbors(total, use_rep="X_scVI")
sc.tl.umap(total, min_dist=0.3)

In [ ]:
sc.pl.umap(
    total,
    color=["dataset"],
    frameon=False,
)

In [ ]:
total.obs